In [1]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
!pip install SpeechRecognition
import speech_recognition as sr
import soundfile
!pip install ffmpeg
!pip install transformers;
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
!gdown https://drive.google.com/uc?id=1GxUl9KfAgn8SKQnuNxDQFX8UAuTcWk1y
!unzip HTNstuff.zip

r = sr.Recognizer()
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=10):
  print('starting')
  display(Javascript(RECORD))
  
  s = output.eval_js('record(%d)' % (sec*1000))
  print('end')
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  
  return 'audio.webm'  # or webm ?

     |████████████████████████████████| 32.8 MB 35 kB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=465df362aae9e679670657065ece407a497293f6011ae7f817abc2437f121e26
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [5]:
record(sec = 5)

starting


<IPython.core.display.Javascript object>

end


'audio.webm'

In [6]:
#Processing from webm to the correct wav format
!ffmpeg -i ./audio.webm -c:a pcm_f32le -y ./out.wav
data, samplerate = soundfile.read('out.wav')
soundfile.write('audio2.wav', data, samplerate, subtype='PCM_16')

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [7]:
with sr.AudioFile('audio2.wav') as source:
  audio_text = r.listen(source)
  text = r.recognize_google(audio_text)
  print(text)

hope you guys have a great day and see you later


In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

thingyID = tokenizer.encode(text, add_special_tokens=True,max_length=256,pad_to_max_length=True)
thingymask = [float(i>0) for i in thingyID]

thingyID = torch.tensor(np.expand_dims(np.array(thingyID), axis = 0))
thingymask = torch.tensor(np.expand_dims(np.array(thingymask), axis = 0))\


int2str = {
    0 : "anger",
    1 : "fear",
    2 : "joy",
    3 : "love",
    4 : "sadness",
    5 : "surprise"
}

model = BertForSequenceClassification.from_pretrained("HTNstuff", num_labels=6)

with torch.no_grad():
    logits = model(thingyID, token_type_ids=None, attention_mask=thingymask)
    logits = logits[0]

    print(int2str[np.argmax(np.array(logits), axis = 1)[0]])

Downloading...
From: https://drive.google.com/uc?id=1jbUW81DmQd9bga3z96qIOx_wJgquf_Oh
To: /content/fineTuneModel.pt
438MB [00:02, 190MB/s]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tensor([2])


TypeError: ignored